# cqapi - Beispiele

## Editor
Der Editor versucht eine nutzerfreundliche Verbindung zur EVA aufzubauen und das Erstellen von Anfragen zu vereinfachen. 
Als erstes wird der Nutzer über die `login`-Methode mit EVA verbunden. Wenn das Fenster "Authentifizierung erfolgreich" erscheint, ist der Editor einsatzbereit

In [ ]:
from cqapi.user_editor import Conquery
eva = Conquery()
eva.login()

### Entdecken von Konzepten, Quellen und Ausgabewerten und deren IDs
Kern der EVA-Anfragen über die cqapi sind die eva-ids. Jedes Element hat eine einzigartige ID.  
Beispiel:  
  * Das Konzept "Alter" hat die ID `alter`
  * das Konzept "Bundesland und Gemeindeschlüssel" die ID `bundesland_und_kgs`

Die Konzepte mit ihren zugehörigen IDs zeigt der Editor über die Methode `show_concepts` an.

In [ ]:
eva.show_concepts()

Die IDs folgen einer hierarchischen Struktur. Die ID eines Kindelements enthält die ID des Elternelements (`<eltern_element_id>.<unterelement_id>`).
Beispiel: 
  * die Quelle "Arzt-Diagnose" des Konzepts "ICD" hat die ID `icd.arzt_diagnose_icd_code`
  * die Quelle "ATC" des Konzepts "ATC" hat die ID `atc.atc`
  * Der Ausgabewert "Liegt vor" des Konzepts "ATC" hat die ID `atc.exists`
  * Der Ausgabewert "Summe DDD" der Quelle "ATC" hat die ID `atc.atc.summe_ddd`
  
Der Großteil der Konzepte hat nur eine Quelle, diese trägt den Namen des Konzepts.  

Die Elemente eines Konzeptes können mit der Methode `show_concept`-Methode angezeigt werden:

In [ ]:
eva.show_concept("atc")

Bei größeren Konzepten können mit dem Parameter `show_all=False` nur die Quellen angegeben werden

In [ ]:
eva.show_concept("icd", show_all=False)

Elemente einer Quelle können über die Methode `show_connector` angezeigt werden

In [ ]:
eva.show_connector("icd.arzt_diagnose_icd_code")

#### Konzepte suchen
Um Unterknoten eines Konzepts zu finden gibt es die Methode `search_concept`

In [ ]:
eva.search_concept("atc", "Memantin")

In [ ]:
eva.search_concept("icd", "Diabetes")

## Anfragen erstellen - das Query Objekt
Mit den richtigen IDs können nun Anfragen erstellt werden. Dafür gibt es die Methode `new_query`, welche EVA-Ids annimmt und ein `Query`-Objekt zurück gibt. Wenn außer der Konzept-ID keine weiteren Parameter angegeben werden, werden die Standard-Einstellungen angewandt. Im Fall von ATC wird die Default-Quelle "ATC" und der Ausgabewert "Liegt vor" ausgewählt. 

In [ ]:
query = eva.new_query(concept_id="atc.n.n06.n06d.n06da.n06da52")

Mit der Methode `show_json`, kann die JSON ausgegeben werden, welche alle Informationen der Anfrage enthält und welche bei einer Ausführung an die EVA geschickt wird

In [ ]:
query.show_json()

Mit den richtigen Parametern können Default-Einstellungen überschrieben werden

In [ ]:
query = eva.new_query(concept_id="atc.n.n06.n06d.n06da.n06da52", select_ids=["atc.atc.summe_ddd"])

In [ ]:
query.show_json()

### Anfragen vereinigen
Anfragen können über die Methode `join_and` und `join_or` verundet bzw. verodert werden.  
Beispiel: Hier verunden wir die Personen, welche das Medikament Memantin bekommen mit dem Personen, die Alzheimer diagnostiziert bekommen haben.

In [ ]:
second_query = eva.new_query(concept_id="icd.e00-e90.e10-e14", 
                                    select_ids=["icd.arzt_diagnose_icd_code.anzahl_arztfaelle"])
query.join_and(second_query)

In [ ]:
query.show_json()

Um das Alter der Personen zu bekommen, vereinigen wir die Anfrage noch mit dem Alter-Konzept

In [ ]:
age_query = eva.new_query(concept_id="alter")
query.join_and(age_query)

Abschließend beschränken wir die Anfrage auf das Jahr 2020

In [ ]:
query.date_restriction(start_date="01.01.2020", end_date="31.12.2020")

### Anfragen ausführen

Um eine Anfrage auszuführen und das Ergebnis zu erhalten gibt es die Methode `get_data`.

In [ ]:
eva.get_data(query)

Für größere Anfragen ist es besser diese erst auszuführen und erst herunterzuladen, wenn die Anfrage fertig berechnet wurde: 

In [ ]:
eva.execute(query)

In [ ]:
eva.check_execution()

In [ ]:
query.download()

## Im Frontend erstellte Anfragen
Über die Methode `from_existing_query` kann über das Label einer bereits erstellt Anfrage die Anfrage geladen werden

In [ ]:
old_query = eva.from_existing_query("test")
eva.download(old_query)

# Export-Formular-Anfragen

In [ ]:
query = eva.new_query(concept_id="atc.n.n06", select_ids=["atc.atc.summe_ddd"])
export_query = eva.new_absolute_export_query(editor_query=query,
                                             start_date="01.01.2020",
                                             end_date="31.12.2020")
eva.add_feature_to_absolute_export_form_query(query=export_query,
                                              concept_id="atc",
                                              select_ids=["atc.atc.anzahl_packungen_select",
                                                          "atc.atc.summe_ddd"])
eva.get_data(export_query)